In this chapter we intend to provide a basic overview of energy consumption evolution in the EU.

We will:

1. See, the evolution of total EU consumption per capita brokendown by energy source
2. See, the evolution of total EU consumption per capita brokendown by consuming sector
3. See, for the last year, the relationship between consumer sector and energy source


In [1]:
from pysdmx.io import get_datasets
import utils
import json
import plotly.express as px


with open('sub_codelists.json', 'r') as f:
    sub_codelists = json.load(f)


We are going to get the data from the Eurostat DB.
We need to get two datasets: The Energy Balances (simplified is enough for this analysis) and the population by country.

For each dataset, we have to provide the list of values we need.
We have created a method that generates automatically the SDMX query from a dictionary of values. We will load the required values from the file with created after the metadata research.

In [3]:
nrg_bal_constraints = {
    'freq': 'A',
    'nrg_bal': sub_codelists['nrg_bal_consumption_basic_plus_other'],
    'siec': sub_codelists['nrg_bal_siec_breakdown'],
    'unit': 'GWH',
    'geo': sub_codelists['geo_total_eu'],
}

demo_pjan_constraints = {
    'freq': 'A',
    'unit': 'NR',
    'age': 'TOTAL',
    'sex': 'T',
    'geo': sub_codelists['geo_total_eu'],
    }


In [4]:
nrg_bal = utils.get_dataset_with_selection('nrg_bal_s', nrg_bal_constraints)
nrg_bal = utils.add_labels(nrg_bal, 'siec')
nrg_bal = utils.add_labels(nrg_bal, 'nrg_bal')

display(nrg_bal.data)

,nrg_bal,siec,TIME_PERIOD,OBS_VALUE,siec_label,nrg_bal_label
0,FC_IND_E,C0000X0350-0370,2000,261061.376,Solid fossil fuels,Final consumption - industry sector - energy use
1,FC_IND_E,C0000X0350-0370,2001,241756.697,Solid fossil fuels,Final consumption - industry sector - energy use
2,FC_IND_E,C0000X0350-0370,2002,218238.569,Solid fossil fuels,Final consumption - industry sector - energy use
3,FC_IND_E,C0000X0350-0370,2003,225650.039,Solid fossil fuels,Final consumption - industry sector - energy use
4,FC_IND_E,C0000X0350-0370,2004,211784.063,Solid fossil fuels,Final consumption - industry sector - energy use
...,...,...,...,...,...,...
1651,FC_TRA_E,W6100_6220,2019,0.000,Non-renewable waste,Final consumption - transport sector - energy use
1652,FC_TRA_E,W6100_6220,2020,0.000,Non-renewable waste,Final consumption - transport sector - energy use
1653,FC_TRA_E,W6100_6220,2021,0.000,Non-renewable waste,Final consumption - transport sector - energy use
1654,FC_TRA_E,W6100_6220,2022,0.000,Non-renewable waste,Final consumption - transport sector - energy use


In [5]:
demo_pjan = utils.get_dataset_with_selection('demo_pjan', demo_pjan_constraints)


# We have to subtract 1 year to the TIME_PERIOD to match the nrg_bal_s_total data
demo_pjan.data.TIME_PERIOD  = demo_pjan.data.TIME_PERIOD  - 1
display(demo_pjan.data)


,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,1999,428473834.0,be
1,2000,429240746.0,b
2,2001,429723142.0,nan
3,2002,431190184.0,nan
4,2003,432762039.0,nan
5,2004,434416272.0,nan
6,2005,435816236.0,nan
7,2006,437227496.0,nan
8,2007,438725386.0,b
9,2008,440047892.0,nan


Preparing a dataset with the consumption per capita in kw/h


In [6]:
full_dataset = nrg_bal.data.merge(demo_pjan.data, on='TIME_PERIOD', how='inner')
full_dataset.rename(columns={'OBS_VALUE_x': 'energy_consumption_gwh', 'OBS_VALUE_y': 'population'}, inplace=True)

display(full_dataset)

,nrg_bal,siec,TIME_PERIOD,energy_consumption_gwh,siec_label,nrg_bal_label,population,OBS_FLAG
0,FC_IND_E,C0000X0350-0370,2000,261061.376,Solid fossil fuels,Final consumption - industry sector - energy use,429240746.0,b
1,FC_IND_E,C0000X0350-0370,2001,241756.697,Solid fossil fuels,Final consumption - industry sector - energy use,429723142.0,nan
2,FC_IND_E,C0000X0350-0370,2002,218238.569,Solid fossil fuels,Final consumption - industry sector - energy use,431190184.0,nan
3,FC_IND_E,C0000X0350-0370,2003,225650.039,Solid fossil fuels,Final consumption - industry sector - energy use,432762039.0,nan
4,FC_IND_E,C0000X0350-0370,2004,211784.063,Solid fossil fuels,Final consumption - industry sector - energy use,434416272.0,nan
...,...,...,...,...,...,...,...,...
1582,FC_TRA_E,W6100_6220,2018,0.000,Non-renewable waste,Final consumption - transport sector - energy use,446135629.0,b
1583,FC_TRA_E,W6100_6220,2019,0.000,Non-renewable waste,Final consumption - transport sector - energy use,447015600.0,e
1584,FC_TRA_E,W6100_6220,2020,0.000,Non-renewable waste,Final consumption - transport sector - energy use,445872542.0,bep
1585,FC_TRA_E,W6100_6220,2021,0.000,Non-renewable waste,Final consumption - transport sector - energy use,445837374.0,ep


In [29]:
def calculate_energy_consumption_per_capita(df):
    df['energy_consumption_PC_kwh'] = df['energy_consumption_gwh'] / df['population'] * 1000000
    return df

data_full_breakdown = nrg_bal.data

data_by_energy_source = nrg_bal.data.groupby(['TIME_PERIOD', 'siec_label'])['OBS_VALUE'].sum().reset_index()
# data_by_energy_source = calculate_energy_consumption_per_capita(data_by_energy_source)

dataset_by_sector = nrg_bal.data.groupby(['TIME_PERIOD', 'nrg_bal_label'])['OBS_VALUE'].sum().reset_index()
# dataset_by_sector = calculate_energy_consumption_per_capita(dataset_by_sector)


In [30]:
fig = px.area(data_by_energy_source, x='TIME_PERIOD', y='OBS_VALUE', color='siec_label')
fig.show()

In [31]:
fig = px.area(dataset_by_sector, x='TIME_PERIOD', y='OBS_VALUE', color='nrg_bal_label')
fig.show()

In [33]:
by_sector_pc =  dataset_by_sector.merge(demo_pjan.data, on='TIME_PERIOD', how='inner')
by_sector_pc.rename(columns={'OBS_VALUE_x': 'energy_consumption_gwh', 'OBS_VALUE_y': 'population'}, inplace=True)

by_sector_pc = calculate_energy_consumption_per_capita(by_sector_pc)

fig = px.area(by_sector_pc, x='TIME_PERIOD', y='energy_consumption_PC_kwh', color='nrg_bal_label')
fig.show()